In [1]:
# Importamos las librerias necesarias

import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')

2023-06-25 18:23:48.179327: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/adrianinfantesromero/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/adrianinfantesromero/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# Cargamos el dataset

train_data = pd.read_csv('CovidNLP/Corona_NLP_train.csv', encoding='latin1')
test_data = pd.read_csv('CovidNLP/Corona_NLP_test.csv', encoding='latin1')

In [3]:
train_data.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41157 entries, 0 to 41156
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   UserName       41157 non-null  int64 
 1   ScreenName     41157 non-null  int64 
 2   Location       32567 non-null  object
 3   TweetAt        41157 non-null  object
 4   OriginalTweet  41157 non-null  object
 5   Sentiment      41157 non-null  object
dtypes: int64(2), object(4)
memory usage: 1.9+ MB


In [5]:
X_train = train_data['OriginalTweet']
y_train = train_data['Sentiment']
X_test = test_data['OriginalTweet']
y_test = test_data['Sentiment']

In [6]:
# Codifica las etiquetas a números enteros
le = LabelEncoder()
train_labels = le.fit_transform(y_train)
test_labels = le.transform(y_test)

In [7]:
# Inicializa el Tokenizer y ajustalo a los datos de entrenamiento
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [8]:
# Transfiere las secuencias de entrenamiento y prueba a secuencias de números
train_sequences = tokenizer.texts_to_sequences(X_train)
test_sequences = tokenizer.texts_to_sequences(X_test)

In [9]:
# Tamaño del vocabulario
vocab_size = len(tokenizer.word_index) + 1

# Longitud máxima de las secuencias
max_length = 100

In [10]:
# Realiza padding a las secuencias
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post', truncating='post')
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post', truncating='post')

In [11]:
# Dimensión de los embeddings y número de unidades LSTM
embedding_dim = 50
lstm_units = 32

# Número de clases
num_classes = len(np.unique(train_labels))

In [12]:
# Convierte las etiquetas a categorías one-hot
train_labels = to_categorical(train_labels, num_classes=num_classes)
test_labels = to_categorical(test_labels, num_classes=num_classes)

In [13]:
# Define el modelo
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
model.add(LSTM(units=64, return_sequences=True))
model.add(LSTM(units=32))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=num_classes, activation='softmax'))

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 50)           4259950   
                                                                 
 lstm (LSTM)                 (None, 100, 64)           29440     
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 dense (Dense)               (None, 32)                1056      
                                                                 
 dense_1 (Dense)             (None, 5)                 165       
                                                                 
Total params: 4,303,027
Trainable params: 4,303,027
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Compila el modelo

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [16]:
# Entrena el modelo

model.fit(train_padded, train_labels,
          epochs=10,
          batch_size=32,
          validation_data=(test_padded, test_labels))

Epoch 1/10
1287/1287 [==============================] - 82s 62ms/step - loss: 1.5770 - accuracy: 0.2766 - val_loss: 1.5835 - val_accuracy: 0.2493
Epoch 2/10
1287/1287 [==============================] - 78s 61ms/step - loss: 1.5758 - accuracy: 0.2775 - val_loss: 1.5854 - val_accuracy: 0.2493
Epoch 3/10
1287/1287 [==============================] - 77s 60ms/step - loss: 1.5757 - accuracy: 0.2775 - val_loss: 1.5853 - val_accuracy: 0.2493
Epoch 4/10
1287/1287 [==============================] - 79s 62ms/step - loss: 1.5754 - accuracy: 0.2775 - val_loss: 1.5834 - val_accuracy: 0.2493
Epoch 5/10
1287/1287 [==============================] - 80s 62ms/step - loss: 1.5755 - accuracy: 0.2775 - val_loss: 1.5830 - val_accuracy: 0.2493
Epoch 6/10
1287/1287 [==============================] - 77s 60ms/step - loss: 1.5754 - accuracy: 0.2775 - val_loss: 1.5836 - val_accuracy: 0.2493
Epoch 7/10
1287/1287 [==============================] - 78s 60ms/step - loss: 1.5754 - accuracy: 0.2775 - val_loss: 1.5856 -

In [17]:
# Evalua el modelo

loss, accuracy = model.evaluate(test_padded, test_labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 24.934176


In [18]:
# Vamos a hacer un grid search para encontrar los mejores parámetros

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

def create_model(lstm_units=50, embedding_dim=50):
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
    model.add(LSTM(units=lstm_units))
    model.add(Dense(units=num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model)

/var/folders/b7/wmcd_tzj3hd8lsdlfgr85gb40000gn/T/ipykernel_4673/2498073969.py:13: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model)


In [19]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'lstm_units': [32, 64, 128],
    'embedding_dim': [50, 100, 200],
    'epochs': [5, 10],
}

In [20]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(train_padded, train_labels)

# Resumen de los resultados
print(f"Mejor: {grid_result.best_score_} usando {grid_result.best_params_}")

Epoch 1/5
858/858 [==============================] - 26s 29ms/step - loss: 1.5727 - accuracy: 0.2827
Epoch 2/5
858/858 [==============================] - 25s 29ms/step - loss: 1.5714 - accuracy: 0.2829
Epoch 3/5
858/858 [==============================] - 25s 29ms/step - loss: 1.5711 - accuracy: 0.2829
Epoch 4/5
858/858 [==============================] - 25s 30ms/step - loss: 1.5712 - accuracy: 0.2829
Epoch 5/5
429/429 [==============================] - 3s 6ms/step - loss: 1.5904 - accuracy: 0.2667
Epoch 1/5
858/858 [==============================] - 26s 29ms/step - loss: 1.5791 - accuracy: 0.2737
Epoch 2/5
858/858 [==============================] - 26s 30ms/step - loss: 1.5783 - accuracy: 0.2752
Epoch 3/5
858/858 [==============================] - 25s 29ms/step - loss: 1.5782 - accuracy: 0.2752
Epoch 4/5
858/858 [==============================] - 25s 29ms/step - loss: 1.5779 - accuracy: 0.2752
Epoch 5/5
429/429 [==============================] - 3s 6ms/step - loss: 1.5712 - accuracy: 0

KeyboardInterrupt: 

In [18]:
# vamos a realizar una predicción

text = "I am so happy"
text = tokenizer.texts_to_sequences([text])
text = pad_sequences(text, maxlen=max_length, padding='post', truncating='post')
prediction = model.predict(text)
print(prediction)

1/1 [==============================] - 1s 510ms/step
[[0.13263637 0.15911351 0.23909669 0.18498784 0.28416556]]


In [19]:
# Vamos a ver la etiqueta predicha

prediction = np.argmax(prediction, axis=-1)
print(prediction)

[4]


In [20]:
# Vamos a ver la etiqueta real

print(le.inverse_transform(prediction))

['Positive']
